In [ ]:
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
#mount my google drive
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision.models as models
from sklearn.model_selection import train_test_split


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. preper Dataset

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Deep Final Work/Training dfs/w2v_train_df.csv')
#train_df = train_df.drop(columns=['Unnamed: 0'])
#train_df.head(5)
train_df = train_df.drop(columns=['ingredients', 'ingredient_embedding'])
train_df

# Split train_df into training and evaluation sets (80/20 split)
train_df, eval_df = train_test_split(train_df, test_size=0.1, random_state=42)
print("Training images:", len(train_df), "Evaluation images:", len(eval_df))


Training images: 2475 Evaluation images: 275


In [ ]:
# Fix the mistake in the 'path' column by replacing "Depp" with "Deep"
train_df['path'] = train_df['path'].str.replace("Depp", "Deep")
eval_df['path']  = eval_df['path'].str.replace("Depp", "Deep")


MSE OR ASE ?

create costum dataset

In [ ]:
class FoodCaloriesDataset(Dataset):
    def __init__(self, df, transform=None):
        """
        Args:
            df (pd.DataFrame): DataFrame with 'path' and 'calories' columns
            transform (callable, optional): Optional transform to be applied
                on a sample image.
        """
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['path']
        # Read the image
        image = Image.open(img_path).convert('RGB')

        # Apply transforms if provided
        if self.transform:
            image = self.transform(image)

        # The label is the calorie count (float)
        label = torch.tensor(row['dish_calories'], dtype=torch.float)

        return image, label


# Typical ImageNet means and stds
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

In [ ]:
# Load the pretrained Inception V3
model = models.inception_v3(pretrained=True)

# The original classifier head outputs 1000 classes for ImageNet
# We'll replace it with a single linear layer for regression
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # single output for calories

# Inception V3 also has an auxiliary output by default.
# If you want to use it during training, you need to handle that.
# The simplest approach: disable the auxiliary by setting:
model.aux_logits = False


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# make train_df['dish_calories'] a int
train_df['dish_calories'] = train_df['dish_calories'].astype(float)
eval_df['dish_calories'] = eval_df['dish_calories'].astype(float)


train_dataset = FoodCaloriesDataset(train_df, transform=train_transform)
val_dataset   = FoodCaloriesDataset(eval_df,   transform=train_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset,   batch_size=16, shuffle=False, num_workers=2)

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# If you have a GPU:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [ ]:
str(train_df['path'][0])

'/content/drive/MyDrive/Deep Final Work/Data/Nutrition5k_RGB/dish_1568318316_rgb.png'

In [ ]:
# num_epochs = 10

# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0

#     for images, labels in train_loader:
#         images = images.to(device)
#         labels = labels.to(device).unsqueeze(1)  # shape (B,1)

#         # Forward pass
#         outputs = model(images)

#         # Compute loss
#         loss = criterion(outputs, labels)

#         # Backprop
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item() * images.size(0)

#     epoch_loss = running_loss / len(train_loader.dataset)

#     # Validation
#     model.eval()
#     val_loss = 0.0
#     with torch.no_grad():
#         for images, labels in val_loader:
#             images = images.to(device)
#             labels = labels.to(device).unsqueeze(1)

#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             val_loss += loss.item() * images.size(0)

#     val_loss /= len(val_loader.dataset)

#     print(f"Epoch [{epoch+1}/{num_epochs}], "
#           f"Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")


Epoch [1/10], Train Loss: 85422.2248, Val Loss: 136686.0862
Epoch [2/10], Train Loss: 76481.6838, Val Loss: 132542.1841
Epoch [3/10], Train Loss: 67857.1678, Val Loss: 114000.5904
Epoch [4/10], Train Loss: 57471.5943, Val Loss: 115451.9963
Epoch [5/10], Train Loss: 46693.1213, Val Loss: 98917.7820
Epoch [6/10], Train Loss: 38116.1578, Val Loss: 102308.6325
Epoch [7/10], Train Loss: 30812.7719, Val Loss: 91678.9394
Epoch [8/10], Train Loss: 25515.3554, Val Loss: 75397.3000
Epoch [9/10], Train Loss: 20665.2034, Val Loss: 76943.6385
Epoch [10/10], Train Loss: 17031.8403, Val Loss: 58045.1555


In [ ]:
# # Define the path where you want to save the model
# save_path = "/content/drive/MyDrive/Deep Final Work/Saved Models/inception_v3_regression.pt"

# # Save the model state dictionary
# torch.save(model, save_path)

# print(f"Model saved to {save_path}")


Model saved to /content/drive/MyDrive/Deep Final Work/Saved Models/inception_v3_regression.pt


In [ ]:
# model.eval()
# test_img_path = "/content/drive/MyDrive/Deep Final Work/Nutrition5k_RGB/dish_1562686492_rgb.png"
# for i in x[:10]:
#   test_img_path = i
#   with torch.no_grad():
#       img = Image.open(test_img_path).convert('RGB')
#       img = train_transform(img).unsqueeze(0).to(device)  # same transforms as training
#       pred_calories = model(img)
#       print("Predicted calories:", pred_calories.item())


NameError: name 'x' is not defined

In [ ]:
eval_df.head(5)

          +---------------------+
          |  Inception (body)  |
          |   (pretrained)     |
          +---------+----------+
                    |
                Feature Map
                    |
               Flatten / GAP
                    |
       +-------------+-------------+
       |  FC1 -> ReLU -> Dropout  |
       +-------------+-------------+
                    |
       +-------------+-------------+
       |  FC2 -> ReLU -> Dropout  |
       +-------------+-------------+
                    |
       +-------------+-------------+
       |        FC3 -> Output     |
       +-------------+-------------+
                    |
               Predicted Calories


In [ ]:
!pip install torch torchvision  # Uncomment if you need to install


In [ ]:

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os

#####################################
# 1. Custom Dataset
#####################################
class FoodCaloriesDataset(Dataset):
    def __init__(self, df, transform=None):
        """
        Args:
            df (pd.DataFrame): Must contain 'path' and 'calories' columns
            transform (callable, optional): Optional transform to be applied
                on an image.
        """
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['path']
        # Read the image
        image = Image.open(img_path).convert('RGB')

        # Apply transforms if provided
        if self.transform:
            image = self.transform(image)

        # The label is the calorie count (float)
        label = torch.tensor(row['dish_calories'], dtype=torch.float)

        return image, label


#####################################
# 2. Transforms
#####################################
# Typical ImageNet means and stds
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((299, 299)),      # for Inception V3
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])


#####################################
# 3. Enhanced Inception Model
#####################################
class InceptionCalorieRegressor(nn.Module):
    """
    An InceptionV3-based model that:
      - Takes an image input
      - Extracts features with the pretrained InceptionV3 body
      - Passes those features through multiple FC layers
      - Outputs a single numeric value (calories)
    """
    def __init__(self, pretrained=True):
        super(InceptionCalorieRegressor, self).__init__()

        # Load the Inception V3 model
        inception = models.inception_v3(pretrained=pretrained)

        # Disable the auxiliary classifier for simplicity
        inception.aux_logits = False

        # Extract the number of input features of the final FC layer
        in_features = inception.fc.in_features

        # Replace the final FC layer with an Identity layer
        # so we can add our own multi-layer head
        inception.fc = nn.Identity()

        self.inception_body = inception

        # multi-layer FC head
        self.fc_head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(512, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(128, 1)  # Single output for calorie regression
        )

    def forward(self, x):
        # Pass the input through Inception (up to the final pool)
        features = self.inception_body(x)  # shape: (batch_size, in_features)

        # Then pass the extracted features through your FC layers
        out = self.fc_head(features)
        return out


#####################################
# 4. Datasets and DataLoaders
#####################################


# Ensure dish_calories are float
train_df['dish_calories'] = train_df['dish_calories'].astype(float)
eval_df['dish_calories']  = eval_df['dish_calories'].astype(float)

train_dataset = FoodCaloriesDataset(train_df, transform=train_transform)
val_dataset   = FoodCaloriesDataset(eval_df,   transform=train_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,  num_workers=2)
val_loader   = DataLoader(val_dataset,   batch_size=16, shuffle=False, num_workers=2)


#####################################
# 5. Initialize Model, Loss, Optimizer
#####################################
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = InceptionCalorieRegressor(pretrained=True)
model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


#####################################
# 6. Training Loop
#####################################
num_epochs = 15

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        # shape (batch_size,) => make it (batch_size,1) for regression
        labels = labels.to(device).unsqueeze(1)

        # Forward pass
        outputs = model(images)  # shape (batch_size, 1)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device).unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

    val_loss /= len(val_loader.dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")




/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/15], Train Loss: 54812.2558, Val Loss: 62752.2488
Epoch [2/15], Train Loss: 12687.3069, Val Loss: 61820.0371
Epoch [3/15], Train Loss: 10247.5480, Val Loss: 59623.7319
Epoch [4/15], Train Loss: 7033.2040, Val Loss: 58756.3218
Epoch [5/15], Train Loss: 7378.1842, Val Loss: 62287.4041
Epoch [6/15], Train Loss: 6137.7094, Val Loss: 59783.3956
Epoch [7/15], Train Loss: 5272.4885, Val Loss: 61877.2815
Epoch [8/15], Train Loss: 4426.9605, Val Loss: 61628.4608
Epoch [9/15], Train Loss: 4492.0447, Val Loss: 60923.6448
Epoch [10/15], Train Loss: 3750.7159, Val Loss: 60097.6902
Epoch [11/15], Train Loss: 3705.3992, Val Loss: 62408.4389
Epoch [12/15], Train Loss: 3137.2733, Val Loss: 58914.4918
Epoch [13/15], Train Loss: 2981.3810, Val Loss: 61287.0541
Epoch [14/15], Train Loss: 3152.3913, Val Loss: 58892.8155
Epoch [15/15], Train Loss: 2765.4417, Val Loss: 62776.7671


In [ ]:
# Define the path where you want to save the ResNet model
save_path = "/content/drive/MyDrive/Deep Final Work/Saved Models/GOOGLES_MODEL_RESORATION.pt"

# Save the model state dictionary
torch.save(model, save_path)

print(f"Model saved to {save_path}")


Model saved to /content/drive/MyDrive/Deep Final Work/Saved Models/GOOGLES_MODEL_RESORATION.pt


In [ ]:
#####################################
# 7. Inference Example
#####################################
model.eval()
eval_paths = eval_df['path'].tolist()
for path in eval_paths[:10]:
  test_img_path = path
  with torch.no_grad():
      img = Image.open(test_img_path).convert('RGB')
      img = train_transform(img).unsqueeze(0).to(device)  # same transforms as training
      pred_calories = model(img)
      print("Predicted calories:", pred_calories.item() ,
            "real caloric value:", eval_df['dish_calories'][eval_df['path']==path])


# ResNet with CBAM

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os

#####################################
# 1. Custom Dataset
#####################################
class FoodCaloriesDataset(Dataset):
    def __init__(self, df, transform=None):
        """
        Args:
            df (pd.DataFrame): Must contain 'path' and 'dish_calories' columns.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['path']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        # Convert dish_calories to a float tensor.
        label = torch.tensor(row['dish_calories'], dtype=torch.float)
        return image, label

#####################################
# 2. Transforms
#####################################
# Standard ImageNet normalization parameters.
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Resize images to 224x224, the typical size for ResNet.
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

#####################################
# 3. CBAM (Convolutional Block Attention Module)
#####################################
class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // reduction, kernel_size=1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_channels // reduction, in_channels, kernel_size=1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        out = self.conv(x_cat)
        return self.sigmoid(out)

class CBAM(nn.Module):
    def __init__(self, in_channels, reduction=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(in_channels, reduction)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        out = x * self.channel_attention(x)
        out = out * self.spatial_attention(out)
        return out

#####################################
# 4. ResNet with CBAM for Calorie Regression
#####################################
class ResNetAttentionCalorieRegressor(nn.Module):
    """
    A ResNet50-based network enhanced with CBAM attention layers.
    The network:
      - Uses a pretrained ResNet50 backbone (up to the last conv layer).
      - Applies CBAM attention to refine the extracted feature maps.
      - Pools the features and passes them through a custom fully connected head.
      - Outputs a single value for calorie regression.
    """
    def __init__(self, pretrained=True):
        super(ResNetAttentionCalorieRegressor, self).__init__()
        # Load a pretrained ResNet50 and remove the average pooling and fc layers.
        resnet = models.resnet50(pretrained=pretrained)
        self.backbone = nn.Sequential(*list(resnet.children())[:-2])  # output shape: (B, 2048, H, W)

        # Add CBAM attention after the backbone.
        self.cbam = CBAM(in_channels=2048, reduction=16, kernel_size=7)

        # Global average pooling to convert (B, 2048, H, W) to (B, 2048, 1, 1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Custom fully connected head.
        self.fc_head = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(128, 1)  # Single output for regression.
        )

    def forward(self, x):
        x = self.backbone(x)      # (B, 2048, H, W)
        x = self.cbam(x)          # Refined with attention.
        x = self.avgpool(x)       # (B, 2048, 1, 1)
        x = torch.flatten(x, 1)   # (B, 2048)
        out = self.fc_head(x)     # (B, 1)
        return out

#####################################
# 5. Datasets and DataLoaders
#####################################
train_df['dish_calories'] = train_df['dish_calories'].astype(float)
eval_df['dish_calories']  = eval_df['dish_calories'].astype(float)

train_dataset = FoodCaloriesDataset(train_df, transform=train_transform)
val_dataset   = FoodCaloriesDataset(eval_df, transform=train_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)

#####################################
# 6. Initialize Model, Loss, Optimizer
#####################################
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNetAttentionCalorieRegressor(pretrained=True)
model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

#####################################
# 7. Training Loop
#####################################
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device).unsqueeze(1)  # Reshape (B,) -> (B, 1)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation phase.
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device).unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

    val_loss /= len(val_loader.dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")

#####################################
# 8. Saving the Model (Optional)
#####################################
save_path = "/content/drive/MyDrive/Deep Final Work/Saved Models/resnet_attention_calorie_regressor.pth"
torch.save(model, save_path)
print("Model saved successfully!")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Train Loss: 49931.8699, Val Loss: 78778.7679
Epoch [2/10], Train Loss: 13324.9813, Val Loss: 61487.1021
Epoch [3/10], Train Loss: 10397.7124, Val Loss: 60759.5048
Epoch [4/10], Train Loss: 8876.3664, Val Loss: 58624.1554
Epoch [5/10], Train Loss: 7646.0468, Val Loss: 60968.0321
Epoch [6/10], Train Loss: 5833.0889, Val Loss: 59260.2897
Epoch [7/10], Train Loss: 6288.3176, Val Loss: 58662.4073
Epoch [8/10], Train Loss: 4844.1687, Val Loss: 58434.0910
Epoch [9/10], Train Loss: 4640.6567, Val Loss: 56882.9093
Epoch [10/10], Train Loss: 4265.1080, Val Loss: 57310.7985
Model saved successfully!


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os

#####################################
# 1. Custom Dataset
#####################################
class FoodCaloriesDataset(Dataset):
    def __init__(self, df, transform=None):
        """
        Args:
            df (pd.DataFrame): Must contain 'path' and 'dish_calories' columns.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['path']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        # Convert dish_calories to a float tensor.
        label = torch.tensor(row['dish_calories'], dtype=torch.float)
        return image, label

#####################################
# 2. Transforms
#####################################
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Standard for ResNet
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

#####################################
# 3. CBAM (Convolutional Block Attention Module)
#####################################
class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // reduction, kernel_size=1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_channels // reduction, in_channels, kernel_size=1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        out = self.conv(x_cat)
        return self.sigmoid(out)

class CBAM(nn.Module):
    def __init__(self, in_channels, reduction=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(in_channels, reduction)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        out = x * self.channel_attention(x)
        out = out * self.spatial_attention(out)
        return out

#####################################
# 4. ResNet with CBAM for Calorie Regression
#####################################
class ResNetAttentionCalorieRegressor(nn.Module):
    def __init__(self, pretrained=True):
        super(ResNetAttentionCalorieRegressor, self).__init__()
        # Load pretrained ResNet50, excluding the final pooling and FC layers.
        resnet = models.resnet50(pretrained=pretrained)
        self.backbone = nn.Sequential(*list(resnet.children())[:-2])  # (B, 2048, H, W)

        # Add CBAM attention module.
        self.cbam = CBAM(in_channels=2048, reduction=16, kernel_size=7)

        # Global average pooling to get (B, 2048, 1, 1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Custom fully connected head.
        self.fc_head = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(128, 1)  # Single output for regression.
        )

    def forward(self, x):
        x = self.backbone(x)      # (B, 2048, H, W)
        x = self.cbam(x)          # Apply attention.
        x = self.avgpool(x)       # (B, 2048, 1, 1)
        x = torch.flatten(x, 1)   # (B, 2048)
        out = self.fc_head(x)     # (B, 1)
        return out

#####################################
# 5. Datasets and DataLoaders
#####################################
# Ensure your train_df and eval_df have 'path' and 'dish_calories' columns.
train_df['dish_calories'] = train_df['dish_calories'].astype(float)
eval_df['dish_calories']  = eval_df['dish_calories'].astype(float)

train_dataset = FoodCaloriesDataset(train_df, transform=train_transform)
# Even if you don't use eval during training, you can still define an eval dataset if needed later.
eval_dataset   = FoodCaloriesDataset(eval_df, transform=train_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)

#####################################
# 6. Initialize Model, Loss, Optimizer
#####################################
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNetAttentionCalorieRegressor(pretrained=True)
model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

#####################################
# 7. Training Loop (No Eval Iteration)
#####################################
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device).unsqueeze(1)  # Reshape to (B, 1)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}")

#####################################
# 8. Saving the Model (Optional)
#####################################
save_path = "/content/drive/MyDrive/Deep Final Work/Saved Models/resnet_attention_calorie_regressor.pth"
torch.save(model, save_path)
print("Model saved successfully!")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Train Loss: 62826.5916
Epoch [2/10], Train Loss: 13298.2276
Epoch [3/10], Train Loss: 11553.0823
Epoch [4/10], Train Loss: 9261.1181
Epoch [5/10], Train Loss: 8149.5509
Epoch [6/10], Train Loss: 5866.4625
Epoch [7/10], Train Loss: 6933.7288
Epoch [8/10], Train Loss: 5706.4182
Epoch [9/10], Train Loss: 5107.4281
Epoch [10/10], Train Loss: 4344.7108
Model saved successfully!


In [ ]:
!pip install torchviz


In [ ]:
from torchviz import make_dot

# Create a dummy input with the shape expected by your model (batch_size=1, channels=3, height=224, width=224)
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# Get the model's output for the dummy input
output = model(dummy_input)

# Generate the computational graph
dot = make_dot(output, params=dict(model.named_parameters()))
dot.format = "png"  # you can change the format as needed
dot.render("resnet_attention_calorie_regressor_graph")

print("Model visualization saved as 'resnet_attention_calorie_regressor_graph.png'")


Model visualization saved as 'resnet_attention_calorie_regressor_graph.png'


In [ ]:
!pip install torchinfo

from torchinfo import summary

model = ResNetAttentionCalorieRegressor(pretrained=True).cuda()
summary(model, input_size=(16, 3, 224, 224))


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Layer (type:depth-idx)                        Output Shape              Param #
ResNetAttentionCalorieRegressor               [16, 1]                   --
├─Sequential: 1-1                             [16, 2048, 7, 7]          --
│    └─Conv2d: 2-1                            [16, 64, 112, 112]        9,408
│    └─BatchNorm2d: 2-2                       [16, 64, 112, 112]        128
│    └─ReLU: 2-3                              [16, 64, 112, 112]        --
│    └─MaxPool2d: 2-4                         [16, 64, 56, 56]          --
│    └─Sequential: 2-5                        [16, 256, 56, 56]         --
│    │    └─Bottleneck: 3-1                   [16, 256, 56, 56]         75,008
│    │    └─Bottleneck: 3-2                   [16, 256, 56, 56]         70,400
│    │    └─Bottleneck: 3-3                   [16, 256, 56, 56]         70,400
│    └─Sequential: 2-6                        [16, 512, 28, 28]         --
│    │    └─Bottleneck: 3-4                   [16, 512, 28, 28]         379,392